In [ ]:
# Check at de rigtige biblioteker er installeret. Og installer dem, hvis det ikke er tilfældet

import sys

try: 
    import requests
    print("requests library has been imported")
except: 
    print("requests library not found. Installing...")
    !pip install requests
    try:
        import requests
    except: 
        print("Something went wrong in the installation of the requests library. Please check your internet connection and consult output from the installation below")

try:
    import pandas
    print("Pandas library has been imported")
except:
    print("Pandas library not found. Installing...")
    !pip install pandas
    
    try:
        import pandas
    except:
        print("Something went wrong in the installation of the Pandas library. Please check your internet connection and consult output from the installation below")

try:
    import bs4
    print("BeatifulSoup library has been imported")
except:
    print("BeatifulSoup library not found. Installing...")
    !pip install beautifulsoup4
    
    try:
        import bs4
    except:
        print("Something went wrong in the installation of the BeatifulSoup library. Please check your internet connection and consult output from the installation below")

try:
    import urllib
    print("Urllib library has been imported")
except:
    print("Urllib library not found. Installing...")
    !pip install urllib
    
    try:
        import urllib
    except:
        print("Something went wrong in the installation of the Urlllib library. Please check your internet connection and consult output from the installation below")

        

## Lad os prøve at scrape noget data direkte fra Wikipedia
Din browser viser indholdet af en hjemmeside baseret på nogle instrukser i HTML-koden. Faktisk kunne man sige, at HTML et semi-struktureret dataformat, der associerer tekst med metadata. For eksempel fortæller HTML-koden din browser hvilke dele af teksten der er brødtekst, hvilke dele der er overskrifter, hvilke dele, der skal rammes ind af en tekstboks, og så videre. 

En scraper udnytter den metadata, der ligger i HTML-koden, til at udvælge og hente betstemte informationer. Vi kan starte med at prøve at hente den fulde HTML-kode fra en af siderne på Wikipedia for at få en idé om, hvordan det ser ud.

In [ ]:
#Hent HTML-koden fra artiklen om Data Science på Wikipedia 

import urllib.request
from bs4 import BeautifulSoup

html_doc = urllib.request.urlopen("https://en.wikipedia.org/wiki/Data_science")

soup = BeautifulSoup(html_doc)
print(soup)


#### Vi kunne for eksempel scrape alle links fra en side
Hvis vi er interesserede i alle hyperlinks vi finder på siden, kan vi bruge HTML-tagget 'href' (hyperlink reference) til udvælge de relevante informationer fra den rå HTML-kode

In [ ]:
#Find de dele af teksten, der er markeret med 'href' (links)
for link in soup.find_all('a'):
    print(link.get('href'))

#### Det kunne også være, vi var interesserede i links fra et bestemt sted på siden
Ved at finde de relvante HTML-tags for den linksamling om 'Machine Learning and Data Sciene' der findes øverst til højre på artikle om Data Science kan vi indsnævre scraperens fokus til kun at hente links herfra.

In [ ]:
#Find linksamlingen om 'Machine Learning and Data Science' og høst links herfra

table = soup.find('table', attrs={'class':'vertical-navbox nowraplinks'})
table_body = table.find('tbody')

for link in table.find_all('a'):
    print(link.get('href'))

## En crawler kan gøre os i stand til at scrape flere sider
Hvis vi vil scrape links fra en masse forskellige sider på Wikipedia, har vi to muligheder. Enten har vi en liste, som vi i forvejen ved er interessant, eller også skal vores scraper være i stand til selv at bevæge sig videre til nye sider, når den finder nogle relevante informationer på den side, den er igang med at høste data fra. Den sidste løsning kaldes 'crawling'. 

Vi kunne for eksmpel bede vores scraper om at starte på artiklen om 'Data Science', lokalisere linksamlingen om 'Machine Learning and Data Science', og så besøge (crawle) alle de sider, der linkes til herfra. Når crawleren besøger de nye sider kan den hente alle de links, den finder hvert sted. 

I princippet kunne vi crawleren fortsætte i uendelighed indtil alle sider på Wikipedia var scrapet - eller alle sider på hele internettet, for den sags skyld. I praksis vil man ofte bestemme, hvor lang snor cralweren for. Dette kaldes 'crawl distance'. I dette tilfælde er crawl distance = 1, fordi vi giver den lov til at tage 1 hyperlink skridt væk fra udgangspunktet. 

In [ ]:
#Crawl de sider, der linkes til i linksamlingen om 'Machine Learning and Data Science' og scrape alle deres links
#Gem resultatet som JSON

data = {}

for link in table.find_all('a'):
    if link.get('href').startswith('/wiki/'):
        html_doc = urllib.request.urlopen('https://en.wikipedia.org'+link.get('href'))
        soup = BeautifulSoup(html_doc)
        data.update({link.get('href'):[]})
        for new_link in soup.find_all('a'):
            if type(new_link.get('href')) is str:
                if new_link.get('href').startswith('/wiki/'):
                    data[link.get('href')].append(new_link.get('href'))

with open('CrawledWikitable_Links.json', 'w',encoding='utf8') as f:
    json.dump(data, f)


In [ ]:
#få overblik over hvilke sider, crawleren har besøgt
data.keys()

In [ ]:
#udforsk links fra en bestemt side
data['/wiki/Computational_learning_theory']

## Hent data der ikke er umiddelbart tilgængelig i HTML-koden via Wikipedias API 
De fleste medier har et Application Programming Interface (API). Det er en portal, der gør det muligt for en applikation at tilgå mediets database automatisk. For at kunne skrive et program, der kan 'ringe op' til en API og bede om bestemte oplysninger, er man nødt til at kende syntaxen. Derudover bestemmer ejeren af APIen hvilke informationer, der er tilgængelige, og på hvilke betingelser. De fleste af API'er har en dokumentation, hvor man kan læse om reglerne.

Vi kunne bede Wikipedias API om informationer, der ikke ummidelbart er tilgængelig for vores scraper i en artikels HTML kode. Det kunne for eksempel handle om, hvordan siden er blevet redigeret. Det er muligt at finde disse informationer andetsteds, men det er praktisk at vi bare kan bede om dem gennem APIen. Dokumentationen findes her: https://www.mediawiki.org/wiki/API:Revisions#API_documentation

Herunder ringer vi op til APIen på adressen http://en.wikipedia.org/w/api.php og beder om informationer ved at bruge nogle af de parametre, der er beskrevet i dokumentationen (også kaldet endpoints).

In [ ]:
#Kald Wikipedias API for at få revisioner på artiklen om 'Data Science'
import requests

S = requests.Session()

URL = "http://en.wikipedia.org/w/api.php"

PARAMS = {
        "action": "query",
        "prop": "revisions",
        "titles": "Data science",
        "rvlimit": "500",
        "rvprop": "timestamp",
        "rvdir": "newer",
        "formatversion": "2",
        "format": "json"

    }
R = S.get(url=URL, params=PARAMS)
DATA = R.json()


In [ ]:
#udforsk resultatet
DATA['query']['pages'][0]

In [ ]:
#Udvid API-kaldet med følgende informationer (listet under 'rvprop' parametret):
#hvilke brugere har lavet hver enkelt revision
#hvor store er revisionerne
#hvilke kommentaerer har brugerne knyttet til revisionerne

import requests

S = requests.Session()

URL = "http://en.wikipedia.org/w/api.php"

PARAMS = {
        "action": "query",
        "prop": "revisions",
        "titles": "Data science",
        "rvlimit": "500",
        "rvprop": "user|timestamp|size|comment",
        "rvdir": "newer",
        "formatversion": "2",
        "format": "json"

    }
R = S.get(url=URL, params=PARAMS)
DATA = R.json()


In [ ]:
#udforsk resultatet
DATA['query']['pages'][0]

## Reverse engineering øvelse
Prøv at åbne det værktøj, der hedder SeeAlsology: https://densitydesign.github.io/strumentalia-seealsology/
SeeAlsology bruger en blanding af scraping, crawling og API-kald til bygge et dataset af Wikipedia artikler op omkring de links, den finder i den såkaldte 'See Also' sektion i bunden af en side. Du kan for eksempel prøve at bruge artiklen om Big Data, der har en fyldig 'See Also' sektion, som input: https://en.wikipedia.org/wiki/Big_data

KAN DU FORKLARE HVORDAN VÆRKTØJET VIRKER?
